<a href="https://colab.research.google.com/github/ccfernandes600/Explorando-IA-Generativa/blob/main/Explorando_IA_Generativa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Explorando IA Generativa em um Pipeline de ETL com Python, projeto feito durante o curso de Ciência de Dados promovido pela DIO.

In [ ]:
#Carregando as bibliotecas necessárias para o projeto
!pip install openai #instaladno a biblioteca da openia
import pandas as pd
import requests
import json
import openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00


In [ ]:
#Montando diretorio  com  o arquivo csv via Google Driver
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# listando o arquivo csv que será usado
!ls '/content/drive/MyDrive/Dados_DIO/'

dados.csv


In [ ]:
# Lendo o arquivo csv com o pandas
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app'

df = pd.read_csv("/content/drive/MyDrive/Dados_DIO/dados.csv")
user_ids = df["UserID"].tolist()
print(user_ids)

def get_user(id):
    response = requests.get(f"{sdw2023_api_url}/users/{id}")
    return response.json() if response.status_code == 200 else None


users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))


[1, 2, 3, 4, 5]


In [ ]:
#TRANSFORM

openai.api_key = ""


def generate_ai_news(user):
    completion = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "Você é um especialista em marketing bancário",
            },
            {
                "role": "user",
                "content": f"Crie uma mensagem para {user['name']}, sobre como juntar pontos no cartão de crédito (máximo de 100 caracteres).",
            },
        ],
    )
    return completion.choices[0].message.content.strip('"')


for user in users:
    news = generate_ai_news(user)
    print(news)
    user["news"].append({"description": news})


In [ ]:
# LOAD
def update_user(user):
    response = requests.put(f"{sdw2023_api_url/{user['id']}}", json=user)
    return True if response.status_code == 200 else False


for user in users:
    success = update_user(user)
    print(f"User {user['name']} updated? {success}!")